## Data Pre-processing

#### Import relevant modules

In [ ]:
#@title Run on TensorFlow 2.x
#%tensorflow_version 2.x
#from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#### Load the dataset

In [ ]:
df = pd.read_csv("weatherAUS.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe(include='all')

In [ ]:
# df.drop(labels = ['Date','Location','Evaporation','Sunshine','Cloud3pm','Cloud9am','RISK_MM'],axis = 1,inplace = True)
df.drop(labels = ['Date','Evaporation','Sunshine','Cloud3pm','Cloud9am','WindDir9am','WindDir3pm','RISK_MM'],axis = 1,inplace = True)

In [ ]:
# Replace Nilai Yes / No
#df['RainToday'].replace({'No':0,'Yes':1},inplace = True) 
#df['RainTomorrow'].replace({'No':0,'Yes':1},inplace = True)
df['RainToday'] = df['RainToday'].map({'Yes': 1, 'No': 0})
df['RainTomorrow'] = df['RainTomorrow'].map({'Yes': 1, 'No': 0})

In [ ]:
#Remove missing values
df.dropna(inplace = True) 
df.shape

In [ ]:
# categorical = ['WindGustDir','WindDir9am','WindDir3pm']
categorical = ['Location','WindGustDir']
df_final = pd.get_dummies(df,columns = categorical,drop_first=True)

In [ ]:
df_final.head()

## Separating Your Training and Testing Datasets

In [ ]:
X = df_final.drop(['RainTomorrow'],axis=1).values
y = df_final['RainTomorrow'].values

In [ ]:
from sklearn.feature_selection import f_regression

In [ ]:
# Since we are more interested in the latter (p-values), we can just take the second array
p_values = f_regression(X,y)[1]
# To be able to quickly evaluate them, we can round the result to 3 digits after the dot
p_values.round(3)

In [ ]:
# list(df_final.columns) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Transforming the Data

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Define the plotting function

In [ ]:
#@title Define the plotting function
def plot_curve(epochs, hist, list_of_metrics):
  """Plot a curve of one or more classification metrics vs. epoch."""  

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Loaded the plot_curve function.")

## Artificial Neural Network

#### Define a deep neural net model

In [ ]:
def create_model(my_learning_rate):
    model = tf.keras.models.Sequential()
    
    # Define the first hidden layer with 10 nodes.
    model.add(tf.keras.layers.Dense(10, kernel_initializer = "uniform",activation = "relu",name='Hidden1'))
    
    # Define a dropout regularization layer. 
    model.add(tf.keras.layers.Dropout(rate=0.1))
    
    # Define the output layer. sigmoid activation
    model.add(tf.keras.layers.Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="binary_crossentropy",
                metrics=["accuracy"])
    
#     model.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    
    return model

#### Define a training function

In [ ]:
def train_model(model, features_arr, label_arr, epochs, 
                batch_size=None):
    history = model.fit(features_arr, label_arr, batch_size, epochs )
    
    # To track the progression of training, gather a snapshot
    # of the model's metrics at each epoch. 
    epochs = history.epoch
    hist = pd.DataFrame(history.history)

    return epochs, hist

#### Call the functions to build and train a deep neural net

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 35
batch_size = 2000

# Establish the model's topography.
my_model = create_model(learning_rate)

# Train the model
epochs, hist = train_model(my_model, X_train, y_train, epochs, 
                          batch_size)
# plot_the_loss_curve(epochs, mse)

# Plot a graph of the metric vs. epochs.
list_of_metrics_to_plot = ['accuracy']
plot_curve(epochs, hist, list_of_metrics_to_plot)

print("\n Evaluate the new model against the test set:")
my_model.evaluate(x = X_test, y = y_test, batch_size=batch_size)

## Running Predictions on the Test Set

In [ ]:
y_pred = my_model.predict(X_test)
y_train_pred = my_model.predict_classes(X_train)

In [ ]:
print('Training Accuracy ---->',accuracy_score(y_train,y_train_pred))
print('Testing Accuracy  ---->',accuracy_score(y_test,y_pred.round()))